# GravixLayer Sandbox – Command Execution + AI Log Summary

This notebook demonstrates a full **log analysis workflow** using:

1. **Command Execution in a sandbox** (via `Sandbox.run_command`)
2. **AI LLM summarization** using `GravixLayer` chat completions
3. **Saving the AI summary** to a local file (`log_ai_summary.txt`) so it can be downloaded

### What we will do

1. Create a sandbox using `Sandbox.create()`
2. Generate a sample `app.log` file (with `INFO` and `ERROR` lines) using shell commands
3. Use commands like `grep`, `wc`, and `cat` to:
   - Extract all `ERROR` lines
   - Count how many errors occurred
4. Send the log data to an **AI model** to get a human-readable summary
5. Save the AI summary to `log_ai_summary.txt` locally, so it can be downloaded from the notebook environment


## 1. Install and Import GravixLayer

We install the GravixLayer SDK if needed and import:

- `Sandbox` for command execution inside an isolated environment
- `GravixLayer` client to call the AI LLM

Make sure you have the environment variable `GRAVIXLAYER_API_KEY` set before running this notebook.

In [1]:
# Install GravixLayer SDK if it's not already installed
!pip install gravixlayer


In [2]:
from gravixlayer import Sandbox, GravixLayer
from textwrap import indent

# Initialize the GravixLayer client for AI calls
client = GravixLayer()

# Choose an AI model
MODEL_NAME = "meta-llama/llama-3.1-8b-instruct"

print("GravixLayer SDK imported. Sandbox and AI client are ready.")

GravixLayer SDK imported. Sandbox and AI client are ready.


## 2. Run Log Processing Commands Inside a Sandbox

In this step, we:

1. Create a sandbox with `Sandbox.create()`
2. Generate a sample `app.log` with `INFO` and `ERROR` lines using a shell command
3. Use `grep` to extract all `ERROR` lines
4. Use `grep -c` to count how many errors occurred
5. Capture the full log and error lines in Python variables for AI

All file operations here are done via **commands inside the sandbox**, not direct Python file I/O.

In [3]:
# These variables will hold data we want to send to the AI later
full_log_text = ""
error_lines_text = ""
error_count_value = None

with Sandbox.create() as sandbox:
    print("Sandbox created. Running log processing commands...\n")

    # 2.1 – Create a sample log file using shell commands
    log_data = """
INFO Server started
INFO Health check OK
ERROR Database connection failed
INFO Retrying connection
ERROR Timeout while querying service
INFO Request completed
ERROR Invalid API key
INFO Background job scheduled
ERROR Disk space critically low
"""

    print("[Step 1] Creating app.log with sample log lines...\n")
    create_log_result = sandbox.run_command(
        "sh",
        ["-c", f"echo '{log_data}' > app.log"]
    )
    print("Create log exit code:", create_log_result.exit_code)

    # 2.2 – Preview the full log file
    print("\n[Step 2] Preview full app.log:\n")
    preview_result = sandbox.run_command("cat", ["app.log"])
    full_log_text = preview_result.stdout
    print(indent(full_log_text, "    "))

    # 2.3 – Find all ERROR lines using grep
    print("[Step 3] Extracting ERROR lines with grep...\n")
    error_lines_result = sandbox.run_command("grep", ["ERROR", "app.log"])
    error_lines_text = error_lines_result.stdout
    print("ERROR lines found:\n")
    print(indent(error_lines_text, "    "))

    # 2.4 – Count number of ERROR entries
    print("[Step 4] Counting number of ERROR entries...\n")
    error_count_result = sandbox.run_command("grep", ["-c", "ERROR", "app.log"])
    error_count_value = error_count_result.stdout.strip()
    print(f"Total ERROR count: {error_count_value}\n")

    # 2.5 – Show final files in the sandbox directory
    print("[Step 5] Files in sandbox (ls -la):\n")
    ls_result = sandbox.run_command("ls", ["-la"])
    print(indent(ls_result.stdout, "    "))

    print("\nLog processing inside sandbox is complete. Sandbox will be cleaned up after this block.")

Sandbox created. Running log processing commands...

[Step 1] Creating app.log with sample log lines...

Create log exit code: 0

[Step 2] Preview full app.log:

     INFO Server started INFO Health check OK ERROR Database connection failed INFO Retrying connection ERROR Timeout while querying service INFO Request completed ERROR Invalid API key INFO Background job scheduled ERROR Disk space critically low 

[Step 3] Extracting ERROR lines with grep...

ERROR lines found:

     INFO Server started INFO Health check OK ERROR Database connection failed INFO Retrying connection ERROR Timeout while querying service INFO Request completed ERROR Invalid API key INFO Background job scheduled ERROR Disk space critically low 

[Step 4] Counting number of ERROR entries...

Total ERROR count: 1

[Step 5] Files in sandbox (ls -la):

    total 3312
    drwxr-xr-x   21 root     root          4096 Dec 10 08:33 .
    drwxr-xr-x   21 root     root          4096 Dec 10 08:33 ..
    -rw-r--r--    1 root 

## 3. Use AI to Summarize the Logs

Now that we have:

- `full_log_text` – the entire log file
- `error_lines_text` – only lines with `ERROR`
- `error_count_value` – how many errors occurred

We send this to an **AI model** through GravixLayer to generate a concise, human-readable summary. This demonstrates how you can plug AI on top of sandbox-generated data.


In [4]:
if not full_log_text:
    raise RuntimeError("full_log_text is empty. Make sure the sandbox cell above ran successfully.")

print("Calling AI model to summarize logs...\n")

prompt_for_ai = f"""
You are a log analysis assistant.

Here is the full application log:
--------------------
{full_log_text}
--------------------

Here are only the lines that contain the word ERROR:
--------------------
{error_lines_text}
--------------------

The total number of ERROR entries detected is: {error_count_value}.

Please provide:
1. A brief summary of what is going wrong in this system
2. The likely root causes based on the errors
3. Suggested next steps for an engineer to investigate
"""

completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful assistant for analyzing server logs."},
        {"role": "user", "content": prompt_for_ai},
    ],
)

ai_summary = completion.choices[0].message.content

print("AI Log Summary:\n")
print(ai_summary)

Calling AI model to summarize logs...

AI Log Summary:

**Summary:**

The system is experiencing several errors, indicating that it is not functioning as expected. The errors are related to database connectivity, service queries, API key validation, and disk space. These issues may be causing the system to malfunction or fail to provide services.

**Likely Root Causes:**

1. **Database Connectivity Issues**: The system is unable to connect to the database, which may be due to network issues, incorrect database credentials, or database server downtime.
2. **Service Query Timeout**: The system is taking too long to query a service, which may be due to slow service response times, high service load, or incorrect query parameters.
3. **Invalid API Key**: The system is unable to validate the API key provided, which may be due to incorrect API key configuration, expired or invalid API keys, or API key validation issues.
4. **Low Disk Space**: The system is running out of disk space, which ma

## 4. Save the AI Summary to a Local File

To show **file handling and download** from the notebook environment, we will:

1. Save the AI-generated summary into a local file named `log_ai_summary.txt`
2. You can then download this file from your notebook / IDE file browser


In [5]:
local_filename = "log_ai_summary.txt"

with open(local_filename, "w", encoding="utf-8") as f:
    f.write("AI Summary for app.log\n")
    f.write("========================\n\n")
    f.write(ai_summary)

print(f"AI summary saved to local file: {local_filename}")
print("You can now download 'log_ai_summary.txt' from the notebook file browser.")

AI summary saved to local file: log_ai_summary.txt
You can now download 'log_ai_summary.txt' from the notebook file browser.


## 5. What This Example Demonstrates

This notebook combines **Sandbox Command Execution** and **AI LLM** usage in a single workflow:

- ✅ Running system commands inside an isolated GravixLayer sandbox
- ✅ Creating and inspecting log files via `sh`, `cat`, `grep`, and `ls`
- ✅ Extracting metrics (error counts) using CLI tools
- ✅ Sending sandbox-generated data to an AI model for analysis
- ✅ Saving the AI output into a local text file for download

You can adapt this pattern for:

- Automated log triage and summarization
- Error trend analysis pipelines
- Health checks with AI-generated human-readable reports
- Any workflow where **sandboxed commands produce data** and **AI explains or enriches it**.
